In [41]:
import numpy as np
from sklearn import tree,metrics,svm,ensemble,neighbors
import pandas as pd
from sklearn.model_selection import train_test_split,KFold
from sklearn import linear_model
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from scipy import optimize

In [108]:
mixture = ['氮化钒铁FeV55N11-A','低铝硅铁','钒氮合金(进口)','钒铁(FeV50-A)','钒铁(FeV50-B)','硅铝钙','硅铝合金FeAl30Si25','硅铝锰合金球',
           '硅锰面（硅锰渣）','硅铁(合格块)','硅铁FeSi75-B','石油焦增碳剂','锰硅合金FeMn64Si27(合格块)','锰硅合金FeMn68Si18(合格块)',
           '碳化硅(55%)','硅钙碳脱氧剂']
columns = ['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si', '钢水净重', '连铸正样C', '连铸正样Mn', '连铸正样S', '连铸正样P', '连铸正样Si', '氮化钒铁FeV55N11-A', '低铝硅铁',
       '钒氮合金(进口)', '钒铁(FeV50-A)', '钒铁(FeV50-B)', '钒铁(FeV50-B).1', '硅铝钙',
       '硅铝合金FeAl30Si25', '硅铝锰合金球', '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B',
       '石油焦增碳剂', '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)',
       '硅钙碳脱氧剂']
x_col = [ '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si','转炉终点温度', '氮化钒铁FeV55N11-A', '低铝硅铁',
       '钒氮合金(进口)', '钒铁(FeV50-A)', '钒铁(FeV50-B)', '硅铝钙',
       '硅铝合金FeAl30Si25', '硅铝锰合金球', '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B',
       '石油焦增碳剂', '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)',
       '硅钙碳脱氧剂']
price = [350000,6500,350000,205000,205000,11800,1000,8500,7600,6000,6000,4600,8150,8150,6100,4000]
len(price)

16

### 计算单位钢水重量加入配料的质量，对最终合金元素浓度的预测模型

In [122]:
data = pd.read_excel('./data1.xlsx')
data.drop(data[data['连铸正样C'].isnull()].index,inplace=True)

# data=data[data['钢号']=='HRB400B    '][columns]
# 离群值处理
outs=['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P',
       '转炉终点Si', '钢水净重', '连铸正样C', '连铸正样Mn', '连铸正样S', '连铸正样P', '连铸正样Si']
for out in outs:
    data.drop(data[(data[out]>(data[out].mean()+3*data[out].std()))|(data[out]<(data[out].mean()-3*data[out].std()))].index,inplace=True)
#单位钢水重量的浓度
for m in mixture:
    data[m]/=data['钢水净重']
data.drop('钢水净重',axis=1,inplace=True)
data.fillna(data.mean(),inplace=True)
data2 = pd.read_excel('data2.xlsx')
data2.head(2)
data['钒铁(FeV50-B)'] = data['钒铁(FeV50-B)']+data['钒铁(FeV50-B).1']
data.drop('钒铁(FeV50-B).1',axis=1,inplace=True)
# X = data[x_col].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)+1e-10)).values
X = data[x_col].values

len(data)#.isna().sum()

703

### 以下分别计算五种元素的预测模型，变量X为1kg钢水质量加入的每种合金配料质量的向量、温度、初始合金元素浓度

In [126]:
Y = data['连铸正样Si'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_Si = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_Si.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index])) # r2系数，平均绝对误差

(0.22295431620669384,
 0.0001457511103253802,
 3.34925406502006e-08,
 0.03952275618782262)

In [127]:
Y = data['连铸正样C'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_C = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_C.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(-0.05049679535047251,
 6.0292927410872176e-05,
 5.0820901488869915e-09,
 0.0239344369690289)

In [128]:
Y = data['连铸正样S'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_S = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_S.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.23421193778366498,
 2.417044177425476e-05,
 9.281691426941879e-10,
 0.06633775505337798)

In [129]:
## S,P缺失值较多，去掉缺失值行后计算模型

In [130]:
data.drop(data[data['转炉终点Mn'].isnull()].index,inplace=True)
data.drop(data[data['转炉终点P'].isnull()].index,inplace=True)

In [151]:
Y = data['连铸正样P'].values
clf = linear_model.LinearRegression()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_P = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_P.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.0561678610716211,
 4.059028229598168e-05,
 2.6184740690369774e-09,
 0.20700194357834176)

In [133]:
Y = data['连铸正样Mn'].values
clf = linear_model.BayesianRidge()
ki = 8
kf = KFold(n_splits=ki)
r2,mae,mse=0,0,0
for train_index,test_index in kf.split(X):
    clf_Mn = clf.fit(X[train_index],Y[train_index])
    pre_y = clf_Mn.predict(X[test_index])
    r2+=metrics.r2_score(Y[test_index],pre_y)
    mae+=metrics.mean_absolute_error(Y[test_index],pre_y)
    mse+=metrics.mean_squared_error(Y[test_index],pre_y)
## r2决定系数，绝对误差，绝对均方误差，绝对误差百分比
r2/ki,mae/ki,mse/ki,np.mean(np.abs((Y[test_index]-pre_y)/Y[test_index]))

(0.06692645855568968,
 0.00024869811477461743,
 1.0125880495885276e-07,
 0.02354684174486633)

## 使用常量替代不是配料的变量，计算一吨合金在下列条件下的最低配料费用

In [134]:
init_ = [data['转炉终点C'].mean(),data['转炉终点Mn'].mean(),data['转炉终点S'].mean(),data['转炉终点P'].mean(),data['转炉终点Si'].mean(),data['转炉终点温度'].mean()]

In [149]:

# a_ = np.array([
#     list(clf_C.coef_[6:]),list(clf_Mn.coef_[6:]),
# #     list(clf_S.coef_[6:]),list(clf_P.coef_[6:])
#     list(clf_Si.coef_[6:])
# ])
# b_ = np.array([
#     0.0023-sum(clf_C.coef_[:6]*init_),
#     0.0145-sum(clf_Mn.coef_[:6]*init_),
# #     0.00045-sum(clf_S.coef_[:6]*init_),
# #     0.00045-sum(clf_P.coef_[:6]*init_),
#     0.0055-sum(clf_Si.coef_[:6]*init_)
# ])


## 建立线性约束不等式以及目标函数,并优化求解

In [155]:

c = data2['价格(元/吨)'].values/1000 #配料价格作为目标函数系数/kg
a = np.array([
    list(clf_C.coef_[6:]),list(-clf_C.coef_[6:]),list(clf_Mn.coef_[6:]),list(-clf_Mn.coef_[6:]),
    list(clf_S.coef_[6:]),list(clf_P.coef_[6:]),
    list(clf_Si.coef_[6:]),list(-clf_Si.coef_[6:])
])
b = np.array([
    0.0025-sum(clf_C.coef_[:6]*init_),-0.0019+sum(clf_C.coef_[:6]*init_),
    0.016-sum(clf_Mn.coef_[:6]*init_),-0.013+sum(clf_Mn.coef_[:6]*init_),
    0.00045-sum(clf_S.coef_[:6]*init_),0.00045-sum(clf_P.coef_[:6]*init_),
    0.0065-sum(clf_Si.coef_[:6]*init_),-0.005+sum(clf_Si.coef_[:6]*init_)
])

#设置配料变量取值范围为0到正无穷
bds = []
for pl in data2['合金配料']:
    bds.append((0,None))
optimize.linprog(c,A_ub=a,b_ub=b,bounds=bds,method='interior-point')

     con: array([], dtype=float64)
     fun: 0.551618108590074
 message: 'Optimization terminated successfully.'
     nit: 12
   slack: array([ 6.00000000e-04, -6.70895631e-14,  3.00000000e-03,  7.49744017e-13,
       -2.57998592e-13,  1.06830422e-03,  1.49999998e-03,  1.54523859e-11])
  status: 0
 success: True
       x: array([7.87862956e-14, 5.64560885e-12, 6.45442466e-14, 1.29857481e-13,
       1.26618061e-13, 2.29688655e-12, 3.71579694e-02, 3.45468334e-12,
       4.18660847e-12, 3.13890342e-12, 2.59407812e-12, 9.19483757e-02,
       3.80533802e-12, 3.93605794e-12, 8.12387566e-03, 1.04854922e-02])

In [154]:
data2

,合金配料,C,Mn,S,P,Si,Al,Ca,V,N,价格(元/吨)
0,氮化钒铁FeV55N11-A,0.000000,0.000,0.0000,0.0000,0.000,0.0000,0.000000,0.560,0.119,350000
1,低铝硅铁,0.000000,0.000,0.0000,0.0000,0.740,0.0008,0.000000,0.000,0.000,6500
2,钒氮合金(进口),0.000000,0.000,0.0000,0.0000,0.000,0.0000,0.000000,0.780,0.151,350000
3,钒铁(FeV50-A),0.003100,0.000,0.0002,0.0006,0.012,0.0000,0.000000,0.512,0.000,205000
4,钒铁(FeV50-B),0.003100,0.000,0.0002,0.0006,0.012,0.0000,0.000000,0.512,0.000,205000
5,硅铝钙,0.000000,0.000,0.0000,0.0000,0.341,0.0910,0.148000,0.000,0.000,11800
6,硅铝合金FeAl30Si25,0.003740,0.000,0.0000,0.0000,0.285,0.3170,0.000000,0.000,0.000,1000
7,硅铝锰合金球,0.000000,0.300,0.0000,0.0000,0.300,0.4000,0.000000,0.000,0.000,8500
8,硅锰面（硅锰渣）,0.017000,0.664,0.0002,0.0018,0.072,0.0000,0.000000,0.000,0.000,7600
9,硅铁(合格块),0.000600,0.000,0.0002,0.0004,0.767,0.0000,0.000000,0.000,0.000,6000


In [160]:
0.000134/100000000

1.34e-12

In [157]:
data[x_col]

,转炉终点C,转炉终点Mn,转炉终点S,转炉终点P,转炉终点Si,转炉终点温度,氮化钒铁FeV55N11-A,低铝硅铁,钒氮合金(进口),钒铁(FeV50-A),...,硅铝合金FeAl30Si25,硅铝锰合金球,硅锰面（硅锰渣）,硅铁(合格块),硅铁FeSi75-B,石油焦增碳剂,锰硅合金FeMn64Si27(合格块),锰硅合金FeMn68Si18(合格块),碳化硅(55%),硅钙碳脱氧剂
0,0.00065,0.001100,0.00030,0.000140,0.004,1644.0,0.000000,0.0,0.000134,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.001142,0.020793,0.000000,0.001183,0.000000
1,0.00077,0.001100,0.00039,0.000210,0.004,1543.0,0.000000,0.0,0.000054,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000916,0.020620,0.000000,0.001779,0.000000
2,0.00035,0.001100,0.00024,0.000200,0.004,1684.0,0.000000,0.0,0.000051,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.001150,0.000000,0.018530,0.001687,0.000000
3,0.00048,0.001200,0.00036,0.000300,0.004,1674.0,0.000000,0.0,0.000054,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.001155,0.000000,0.019701,0.001793,0.000000
4,0.00036,0.000900,0.00028,0.000250,0.004,1800.0,0.000000,0.0,0.000055,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.001243,0.000000,0.020028,0.001823,0.000000
5,0.00022,0.000700,0.00038,0.000140,0.004,1660.0,0.000000,0.0,0.000057,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.001279,0.000000,0.020611,0.001876,0.000000
6,0.00070,0.001200,0.00020,0.000130,0.004,1654.0,0.000000,0.0,0.000064,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000746,0.000000,0.019280,0.001697,0.000000
7,0.00063,0.001400,0.00025,0.000310,0.004,1666.0,0.000000,0.0,0.000055,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.001095,0.000000,0.019849,0.001807,0.000000
8,0.00106,0.000900,0.00022,0.000120,0.004,1666.0,0.000000,0.0,0.000067,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000781,0.000000,0.021265,0.001777,0.000000
9,0.00043,0.001100,0.00028,0.000210,0.004,1658.0,0.000000,0.0,0.000067,0.000000,...,0.0,0.0,0.002939,0.0,0.0,0.001136,0.000000,0.017769,0.001764,0.000000
